In [1]:
import numpy as np
import pandas as pd

# Step One: Collect the data

I queried the Census Website's "American Fact Finder" for income data from the American Community Survey (ACS). You can find that data (for the District of Columbia) here: http://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?pid=ACS_14_5YR_B19001&prodType=table

I chose the option from the interface to download the table as a .csv file, which we load in to pandas right here!

In [2]:
df = pd.read_csv("ACS_14_5YR_B19001_with_ann.csv")

Below, I show the first row of the data-frame (transposed for ease of viewing), which is a detailed name for each of the cryptic ACS column names. I'll use this for reference as I'm exploring the data, and will mention it several times in this notebook.

In [3]:
df.head(1).T

,0
GEO.id,Id
GEO.id2,Id2
GEO.display-label,Geography
HD01_VD01,Estimate; Total:
HD02_VD01,Margin of Error; Total:
HD01_VD02,"Estimate; Total: - Less than $10,000"
HD02_VD02,"Margin of Error; Total: - Less than $10,000"
HD01_VD03,"Estimate; Total: - $10,000 to $14,999"
HD02_VD03,"Margin of Error; Total: - $10,000 to $14,999"
HD01_VD04,"Estimate; Total: - $15,000 to $19,999"


Next, I drop the first row of the dataframe, which will not be useful as I actually begin to perform actions on the data. On the line after I drop the row, I call ```df.head()``` to ensure that the row is gone.

In [4]:
df = df.drop(0)

In [5]:
df.head(1).T

,1
GEO.id,1500000US110010001001
GEO.id2,110010001001
GEO.display-label,"Block Group 1, Census Tract 1, District of Col..."
HD01_VD01,641
HD02_VD01,104
HD01_VD02,25
HD02_VD02,40
HD01_VD03,20
HD02_VD03,31
HD01_VD04,0


Great, it worked! Now, lets find out some information about our data-set.

In [6]:
df.HD01_VD01.astype(int).sum()

267415

The total number of data-points collected in this survey was apparently 267,415. I arrived at that number by finding the sum of the cells in the HD01_VD01 column which, according to the detailed column-names row from code-block 3 of this notebook, is the total-population surveyed in this particular years ACS. This is useful to know because when we finally move on to mapping this data, THAT will be the number of points we should see.

Speaking of mapping, now is a great time to explain how the Census Bureau breaks down the country into geo-spatial sections. This is essential knowledge for anybody who wants to visualize census data on a map! In short, the Census Buraeu breaks down each of the 50 stats (as well as Puerto Rico and The District of Columbia) into six additional subgroups. Those subgroups, in ascending degree of specificitiy, are County -> County Subdivision -> Place -> Place Part -> Census Tract -> Block Group -> Block, where states are made of counties, which are made of county subdivions... you see where this is going. In the style of a directory hierarchy, these groups would look like this:

    |State  
    |-->County
    |   |-->County Subdivision
    |   |   |-->Place
    |   |   |   |-->Place Part
    |   |   |   |   |-->Census Tract
    |   |   |   |   |   |-->Block Group
    |   |   |   |   |   |   |-->Block

Now that the lesson pn Census organization techinqiues is over, it's time to get back to the data... which brings up a common problem-- we need more of it! The data in the .csv than I downloaded from American Factfinder separates data based on block-group, but I currently have no way to associate those block-groups with coordinates on a map. 

Luckily, TIGER shapefiles exist! TIGER stands for 'Topilogically Integrated Geographic Encoding and Referencing', and a TIGER product is exactly what we need right now.  We can find a .shp file for Washington, D.C. block-groups at http://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2015&layergroup=Block+Groups

I'll be using several libraries to work with this .shp file. First, I use `ogr` from `osgeo` to load in the .shp file. I also import certain shapely functions for working with individual polygons within the .shp file (but more on that later!)

In [7]:
from osgeo import ogr
from shapely.wkb import loads
from shapely.geometry import *
from random import uniform

from globalmaptiles import GlobalMercator

You'll also see that I importet from a library called globalmaptiles. I need the GlobalMercator class because, in order to visualize the data using the tool I want to use (DataShader), I need to project geodetic coordinates as WebMercator. This class allows me to do that.

In [8]:
# Create a GlobalMercator object for later conversions    
merc = GlobalMercator()

# Open .shp file downloaded from TIGER
ds = ogr.Open(r"/home/terminal/projects/tl_2014_11_bg.shp", 0)

**IMPORTANT NOTE:** In order to load this .shp file using ogr, you need to have all files extracted from the .zip that you downloaded from TIGER in the same directory. I don't know why, but that's how it's gotta be.

# Step Two: Transform data into a form that can be mapped using DataShader. 

In this section of the notebook, I will be *heavily* borrowing from the fantastic code written by unorthodox123 in their RacialDotMap repository on GitHub. [Their racial dot map](http://demographics.coopercenter.org/DotMap/index.html)  was part of the inspiration for me creating the visualization that I'm working towards in this notebook. You can find this repository [here](https://github.com/unorthodox123/RacialDotMap).

First things first, I'm going to get a ogr.Layer object from the .shp file we loaded using ogr, then save it to the variable `lyr`. After that, I call `lyr.ResetReading()` to reset the object to the first feature in the layer. 

After that, the code uses `lyr.GetLayerDefn()` to get the scheme information for the layer. It then uses that information to create a list of each field in the schema.

In [9]:
 # Obtain the first (and only) layer in the shapefile
lyr = ds.GetLayerByIndex(0)

lyr.ResetReading()

# Obtain the field definitions in the shapefile layer

feat_defn = lyr.GetLayerDefn()
field_defns = [feat_defn.GetFieldDefn(i) for i in range(feat_defn.GetFieldCount())]

Next, I loop through `field_defns` in order to get the index for `'GEOID'`. I need this in order to correlate block-groups from my ACS data with block-group polygons from my TIGER data.

In [10]:
# Obtain the index of the field for the count for GEOID

for i, defn in enumerate(field_defns):
    if defn.GetName() == 'GEOID':
        geo_field = i

Up next comes the real heavy-lifting of this notebook. It's a long (un-refactored or optimized) block of code, so I'll explain here what it's doing.

Basically, my end goal is to create a geospatial representation of income data for Washington, D.C.. I have data organized for each block-group in D.C., but simply shading in a polygon isn't good enough for me. Therefore, I'm going to randomly distribute a number of points (within each TIGER polygon) equal to the number of points for each income-bracket listed in our ACS dataframe. I'm going to save these points in a pandas dataframe that will eventually be exported into a .csv. 

In [ ]:
n_features = len(lyr)
out_csv = pd.DataFrame(columns=('GEO.id2', 'x', 'y', 'quadkey', 'income_category'))
for j, feat in enumerate( lyr ):

    # Print a progress read-out for every 10 features

    if j % 10 == 0:
        print("{0}/{1}".format(j+1,n_features))
        print(out_csv.count())

    # Obtain total population and income bracket count for each block group
    geo = feat.GetField(geo_field)
    total_pop = int(df[df['GEO.id2'] == geo].HD01_VD01)
    lt_10k = int(df[df['GEO.id2'] == geo].HD01_VD02)
    bt_10k_15k = int(df[df['GEO.id2'] == geo].HD01_VD03)
    bt_15k_20k = int(df[df['GEO.id2'] == geo].HD01_VD04)
    bt_20k_25k = int(df[df['GEO.id2'] == geo].HD01_VD05)
    bt_25k_30k = int(df[df['GEO.id2'] == geo].HD01_VD06)
    bt_30k_35k = int(df[df['GEO.id2'] == geo].HD01_VD07)
    bt_35k_40k = int(df[df['GEO.id2'] == geo].HD01_VD08)
    bt_40k_45k = int(df[df['GEO.id2'] == geo].HD01_VD09)
    bt_45k_50k = int(df[df['GEO.id2'] == geo].HD01_VD10)
    bt_50k_60k = int(df[df['GEO.id2'] == geo].HD01_VD11)
    bt_60k_75k = int(df[df['GEO.id2'] == geo].HD01_VD12)
    bt_75k_100k = int(df[df['GEO.id2'] == geo].HD01_VD13)
    bt_100k_125k = int(df[df['GEO.id2'] == geo].HD01_VD14)
    bt_125k_150k = int(df[df['GEO.id2'] == geo].HD01_VD15)
    bt_150k_200k = int(df[df['GEO.id2'] == geo].HD01_VD16)
    gt_200k = int(df[df['GEO.id2'] == geo].HD01_VD17)

    # Obtain the OGR polygon object from the feature

    geom = feat.GetGeometryRef()

    if geom is None:
        print("continued")
        continue

    # Convert the OGR Polygon into a Shapely Polygon

    poly = loads(geom.ExportToWkb())

    if poly is None:
        continue
    bbox = poly.bounds

    if not bbox:
        continue

    leftmost,bottommost,rightmost,topmost = bbox
    
    # Generate a point object within the census block for every person by income category   
    for i in range(lt_10k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD02'         

        # Append data to dataframe
        out_csv = out_csv.append(dict([('GEO.id2',geo),
                                       ('x',x),
                                       ('y',y),
                                       ('quadkey',quadkey),
                                       ('income_category',income_category)]),
                                 ignore_index=True)
        
    for i in range(bt_10k_15k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD03'         

        # Append data to dataframe
        out_csv = out_csv.append(dict([('GEO.id2',geo),
                                       ('x',x),
                                       ('y',y),
                                       ('quadkey',quadkey),
                                       ('income_category',income_category)]),
                                 ignore_index=True)
        
    for i in range(bt_15k_20k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD04'         

        # Append data to dataframe

        out_csv = out_csv.append(dict([('GEO.id2',geo),
                                       ('x',x),
                                       ('y',y),
                                       ('quadkey',quadkey),
                                       ('income_category',income_category)]),
                                 ignore_index=True)
        
    for i in range(bt_20k_25k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD05'         

        # Append data to dataframe
        out_csv = out_csv.append(dict([('GEO.id2',geo),
                                       ('x',x),
                                       ('y',y),
                                       ('quadkey',quadkey),
                                       ('income_category',income_category)]),
                                 ignore_index=True)
        
    for i in range(bt_25k_30k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD06'         

        # Append data to dataframe
        out_csv = out_csv.append(dict([('GEO.id2',geo),
                                       ('x',x),
                                       ('y',y),
                                       ('quadkey',quadkey),
                                       ('income_category',income_category)]),
                                 ignore_index=True)
        
    for i in range(bt_30k_35k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD07'         

        # Append data to dataframe
        out_csv = out_csv.append(dict([('GEO.id2',geo),
                                       ('x',x),
                                       ('y',y),
                                       ('quadkey',quadkey),
                                       ('income_category',income_category)]),
                                 ignore_index=True)
        
    for i in range(bt_35k_40k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD08'         

        # Append data to dataframe
        out_csv = out_csv.append(dict([('GEO.id2',geo),
                                       ('x',x),
                                       ('y',y),
                                       ('quadkey',quadkey),
                                       ('income_category',income_category)]),
                                 ignore_index=True)
        
    for i in range(bt_40k_45k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD09'         

        # Append data to dataframe
        out_csv = out_csv.append(dict([('GEO.id2',geo),
                                       ('x',x),
                                       ('y',y),
                                       ('quadkey',quadkey),
                                       ('income_category',income_category)]),
                                 ignore_index=True)
        
    for i in range(bt_45k_50k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD10'         

        # Append data to dataframe
        out_csv = out_csv.append(dict([('GEO.id2',geo),
                                       ('x',x),
                                       ('y',y),
                                       ('quadkey',quadkey),
                                       ('income_category',income_category)]),
                                 ignore_index=True)
        
    for i in range(bt_50k_60k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD11'         

        # Append data to dataframe
        out_csv = out_csv.append(dict([('GEO.id2',geo),
                                       ('x',x),
                                       ('y',y),
                                       ('quadkey',quadkey),
                                       ('income_category',income_category)]),
                                 ignore_index=True)
        
    for i in range(bt_60k_75k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD12'         

        # Append data to dataframe
        out_csv = out_csv.append(dict([('GEO.id2',geo),
                                       ('x',x),
                                       ('y',y),
                                       ('quadkey',quadkey),
                                       ('income_category',income_category)]),
                                 ignore_index=True)
        
    for i in range(bt_75k_100k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD13'         

        # Append data to dataframe
        out_csv = out_csv.append(dict([('GEO.id2',geo),
                                       ('x',x),
                                       ('y',y),
                                       ('quadkey',quadkey),
                                       ('income_category',income_category)]),
                                 ignore_index=True)
        
    for i in range(bt_100k_125k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD14'         

        # Append data to dataframe
        out_csv = out_csv.append(dict([('GEO.id2',geo),
                                       ('x',x),
                                       ('y',y),
                                       ('quadkey',quadkey),
                                       ('income_category',income_category)]),
                                 ignore_index=True)
        
    for i in range(bt_125k_150k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD15'         

        # Append data to dataframe
        out_csv = out_csv.append(dict([('GEO.id2',geo),
                                       ('x',x),
                                       ('y',y),
                                       ('quadkey',quadkey),
                                       ('income_category',income_category)]),
                                 ignore_index=True)
        
    for i in range(bt_150k_200k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD16'         

        # Append data to dataframe
        out_csv = out_csv.append(dict([('GEO.id2',geo),
                                       ('x',x),
                                       ('y',y),
                                       ('quadkey',quadkey),
                                       ('income_category',income_category)]),
                                 ignore_index=True)
        
    for i in range(gt_200k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD17'         

        # Append data to dataframe
        out_csv = out_csv.append(dict([('GEO.id2',geo),
                                       ('x',x),
                                       ('y',y),
                                       ('quadkey',quadkey),
                                       ('income_category',income_category)]),
                                 ignore_index=True)

I'm not going to run the above code because

    1) I already have, and
    2) It took about 3 hours
    
I do plan on coming back to this on a later post and examining how I couldve generated all of these random coordinates faster and cleaner, but that will have to wait. I really want to see this map!

Now, export this dataframe to a .csv for further use (and to avoid running that code again...), and we have created our dataset! Now, we just need to get the appropriate visualization libraries and we can see it in action!

In [ ]:
out_csv.to_csv("censusBlock_incomePoints.csv")

That's it for this post! If you have any comments, questions, or you just want to get in touch, please email me at `codebycandlelight@gmail.com`. I hope to hear from some of you! See you next time!